In [1]:
import sys
sys.path.append("../")

import argparse
import datetime
import json
import os
import re
from time import sleep

import numpy as np
import pandas as pd
import requests

from vook_db_v7.config import MAX_PAGE, REQ_URL, WANT_ITEMS, req_params

# from logzero import logger

In [2]:
#apiコールした結果からdataframeを出力する関数を定義
def DataFrame_maker(keyword, platform_id, knowledge_id, size_id):
    cnt = 1
    df = pd.DataFrame(columns=WANT_ITEMS)
    req_params["page"] = cnt
    req_params["keyword"] = keyword
    while True:
        req_params["page"] = cnt
        res = requests.get(REQ_URL, req_params)
        res_code = res.status_code
        res = json.loads(res.text)
        if res_code != 200:
            print(
                f"""
            ErrorCode -> {res_code}\n
            Error -> {res['error']}\n
            Page -> {cnt}"""
            )
        else:
            if res["hits"] == 0:
                print("返ってきた商品数の数が0なので、ループ終了")
                break
            tmp_df = pd.DataFrame(res["Items"])[WANT_ITEMS]
            df = pd.concat([df, tmp_df], ignore_index=True)
        if cnt == MAX_PAGE:
            print("MAX PAGEに到達したので、ループ終了")
            break
        # logger.info(f"{cnt} end!")
        cnt += 1
        # リクエスト制限回避
        sleep(1)
    
        print("Finished!!")
    df["platform_id"] = platform_id
    df["knowledge_id"] = knowledge_id
    df["size_id"] = size_id
    df_main = df.rename(
        columns={"itemName": "name", "itemPrice": "price", "itemUrl": "url"}
    )
    df_main = df_main.reindex(
        columns=["id", "name", "url", "price", "knowledge_id", "platform_id", "size_id"]
    )
    run_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
    df_main["created_at"] = run_time
    df_main["updated_at"] = run_time
    return df_main

In [3]:
platform_id = 1
size_id = 999

brand = "リーバイス levis"
item_1, knowledge_id_1 = "501xx 片面タブ", 11
item_2, knowledge_id_2 = "501xx 両面タブ", 12

list_item = [item_1,item_2]
list_knowledge_id = [knowledge_id_1, knowledge_id_2]

n_bulk = len(list_item) #一括で処理する検索キーワードパターンの数

'''以下、一括処理を行う'''

#空データフレーム作成
df_bulk = pd.DataFrame()
columns=["id", "name", "url", "price", "knowledge_id", "platform_id", "size_id"]
df_bulk[columns] = "","", "", "", "", "", ""

for n in np.arange(n_bulk):
    keyword = f"{brand} {list_item[n]} 中古"
    knowledge_id = list_knowledge_id[n]
    print('検索キーワード:['+keyword+']', 'knowledge_id:',knowledge_id)
    output = DataFrame_maker(keyword, platform_id, knowledge_id, size_id)
    df_bulk = pd.concat([df_bulk, output],ignore_index=True)

#一括処理して出来上がったdataframeに対して最後にidを振る
df_prev = pd.read_csv("../data/output/products_raw_prev.csv")
PREV_ID_MAX = df_prev["id"].max()
df_bulk["id"] = np.arange(PREV_ID_MAX, PREV_ID_MAX + len(df_bulk)) + 1

検索キーワード:[リーバイス levis 501xx 片面タブ 中古] knowledge_id: 11
Finished!!
返ってきた商品数の数が0なので、ループ終了
検索キーワード:[リーバイス levis 501xx 両面タブ 中古] knowledge_id: 12
Finished!!
返ってきた商品数の数が0なので、ループ終了


In [4]:
file_name = "products_raw"
df_bulk.to_csv("../data/output/" + file_name + ".csv", index=False)